In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
from collections import deque
import numpy as np
import sys
import os
from tqdm import tqdm
import pickle

# Add the project root directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

from src.ParObsSnakeEnv import ParObsSnakeEnv
from src.FullObsSnakeEnv import FullObsSnakeEnv
from src.utils import compute_metrics
import matplotlib.pyplot as plt

In [2]:
class DQN(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=128):
        super(DQN, self).__init__()
        self.hidden_dim = hidden_dim
        self.fc1 = nn.Linear(input_dim, self.hidden_dim)
        self.fc2 = nn.Linear(self.hidden_dim, self.hidden_dim)
        self.fc3 = nn.Linear(self.hidden_dim, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [3]:
# Define the DQN agent
class DQNAgent:
    def __init__(self, state_dim, action_dim, lr=0.001, gamma=0.99, epsilon=1.0, epsilon_decay=0.995, epsilon_min=0.01, memory_size=10000, batch_size=64):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        self.batch_size = batch_size

        self.memory = deque(maxlen=memory_size)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = DQN(state_dim, action_dim).to(self.device)
        self.target_model = DQN(state_dim, action_dim).to(self.device)
        self.target_model.load_state_dict(self.model.state_dict())
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)
        self.criterion = nn.MSELoss()

        # Metrics for plotting
        self.episode_rewards = []
        self.episode_losses = []
        self.epsilon_values = []
        self.average_q_values = []

    def choose_action(self, state):
        if random.random() < self.epsilon:
            return random.randint(0, self.action_dim - 1), None
        state = torch.FloatTensor(state).unsqueeze(0).to(self.device)
        state = state.reshape((1, -1))
        with torch.no_grad():
            # print(">>>>", state.shape)
            q_values = self.model(state)
        return torch.argmax(q_values).item(), None

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def replay(self):
        if len(self.memory) < self.batch_size:
            return

        batch = random.sample(self.memory, self.batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)

        states = torch.FloatTensor(states).to(self.device)
        actions = torch.LongTensor(actions).to(self.device)
        rewards = torch.FloatTensor(rewards).to(self.device)
        next_states = torch.FloatTensor(next_states).to(self.device)
        dones = torch.FloatTensor(dones).to(self.device)

        states = states.reshape((self.batch_size, -1))
        next_states = next_states.reshape((self.batch_size, -1))

        # Compute current Q-values
        q_values = self.model(states)
        q_values = q_values.gather(1, actions.unsqueeze(1)).squeeze(1)

        # Compute target Q-values
        next_q_values = self.target_model(next_states).max(1)[0]
        target_q_values = rewards + (1 - dones) * self.gamma * next_q_values

        # Update the Q-network
        loss = self.criterion(q_values, target_q_values)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # Log the loss
        self.episode_losses.append(loss.item())

        # Track average Q-value
        self.average_q_values.append(q_values.mean().item())

        # Decay epsilon
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

    def update_target_model(self):
        self.target_model.load_state_dict(self.model.state_dict())

    def save(self, filename):
        """Saves the entire agent to a file with all tensors moved to CPU."""
        state = {
            'model_state_dict': self.model.to('cpu').state_dict(),
            'target_model_state_dict': self.target_model.to('cpu').state_dict(),
            'optimizer_state_dict': {k: v for k, v in self.optimizer.state_dict().items()},
            'hyperparameters': {
                'state_dim': self.state_dim,
                'action_dim': self.action_dim,
                'gamma': self.gamma,
                'epsilon': self.epsilon,
                'epsilon_decay': self.epsilon_decay,
                'epsilon_min': self.epsilon_min,
                'batch_size': self.batch_size,
            },
            'memory': list(self.memory),  # Convert deque to list
        }
        with open(filename, 'wb') as f:
            pickle.dump(state, f)
        print(f"Agent saved to {filename}")

        # Move the models back to their original device if needed
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(device)
        self.target_model.to(device)
    
    def load(cls, filename, lr=0.001, device=None):
        """Loads the agent from a file."""
        with open(filename, 'rb') as f:
            state = pickle.load(f)

        # Determine the device to load the model onto
        if device is None:
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Recreate the agent
        agent = cls(
            state['hyperparameters']['state_dim'],
            state['hyperparameters']['action_dim'],
            lr=lr,
            gamma=state['hyperparameters']['gamma'],
            epsilon=state['hyperparameters']['epsilon'],
            epsilon_decay=state['hyperparameters']['epsilon_decay'],
            epsilon_min=state['hyperparameters']['epsilon_min'],
            batch_size=state['hyperparameters']['batch_size'],
        )

        # Restore the agent's state
        agent.model.load_state_dict(state['model_state_dict'])
        agent.target_model.load_state_dict(state['target_model_state_dict'])
        agent.optimizer.load_state_dict(state['optimizer_state_dict'])
        agent.memory = deque(state['memory'], maxlen=len(state['memory']))

        # Move the model to the specified device
        agent.model = agent.model.to(device)
        agent.target_model = agent.target_model.to(device)

        print(f"Agent loaded from {filename} onto device: {device}")
        return agent

    # Train the agent
    def train(self, env, episodes=1000, update_target_every=10, save_plots=False, plots_path='dqn_training_plots.png'):
        for episode in tqdm(range(episodes), desc="Training", unit='episode'):
            state = env.reset()
            total_reward = 0
            done = False

            while not done:
                action, _ = self.choose_action(state)
                next_state, reward, done, _ = env.step(action)
                self.remember(state, action, reward, next_state, done)
                self.replay()
                state = next_state
                total_reward += reward

            self.episode_rewards.append(total_reward)
            self.epsilon_values.append(self.epsilon)

            if (episode + 1) % update_target_every == 0:
                self.update_target_model()

        if save_plots:
            self.save_plots(plots_path)
    
    def save_plots(self, plots_path):
        plots_dir = os.path.dirname(plots_path)
        os.makedirs(plots_dir, exist_ok=True)

        fig, axs = plt.subplots(2, 2, figsize=(15, 10))

        # Rewards per episode
        axs[0, 0].plot(self.episode_rewards)
        axs[0, 0].set_title("Episode Rewards")
        axs[0, 0].set_xlabel("Episode")
        axs[0, 0].set_ylabel("Total Reward")

        # Loss per episode
        axs[0, 1].plot(self.episode_losses)
        axs[0, 1].set_title("Loss Over Training")
        axs[0, 1].set_xlabel("Episode")
        axs[0, 1].set_ylabel("Loss")

        # Epsilon decay
        axs[1, 0].plot(self.epsilon_values)
        axs[1, 0].set_title("Epsilon Decay")
        axs[1, 0].set_xlabel("Episode")
        axs[1, 0].set_ylabel("Epsilon Value")

        # Average Q-values
        axs[1, 1].plot(self.average_q_values)
        axs[1, 1].set_title("Average Q-Values")
        axs[1, 1].set_xlabel("Episode")
        axs[1, 1].set_ylabel("Average Q-Value")

        plt.tight_layout()
        plt.savefig(plots_path)
        plt.close()

In [4]:
grid_size = 10
# env = FullObsSnakeEnv(grid_size=grid_size, interact=False)
env = ParObsSnakeEnv(grid_size=grid_size, interact=False)
if isinstance(env, FullObsSnakeEnv):
    state_dim = env.observation_space.shape[0] * env.observation_space.shape[1] * env.observation_space.shape[2]
else:
    state_dim = env.observation_space.shape[0]

action_dim = env.action_space.n
agent = DQNAgent(state_dim, action_dim)

In [5]:
num_episodes = 300
agent.train(env, episodes=num_episodes)

Training:   0%|          | 0/300 [00:00<?, ?episode/s]/var/folders/md/5b3wx8h566j7t6cyr60qt_r40000gn/T/ipykernel_70204/2397199468.py:46: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:278.)
  states = torch.FloatTensor(states).to(self.device)
Training: 100%|██████████| 300/300 [01:13<00:00,  4.10episode/s]


In [6]:
environment = 'full 'if isinstance(env, FullObsSnakeEnv) else 'par'

agent_name = f'dqn_agent_{environment}_{num_episodes}_{grid_size}.pkl'
model_weights_dir = os.path.join('../..', 'models', 'dqn')
os.makedirs(model_weights_dir, exist_ok=True)
agent_path = os.path.join(model_weights_dir, agent_name)

agent.save(agent_path)
# agent = DQNAgent.load(agent_path)

Agent saved to ../../models/dqn/dqn_agent_par_300_10.pkl


In [7]:
if isinstance(env, ParObsSnakeEnv):
    env = ParObsSnakeEnv(grid_size=2*grid_size, interact=False)

model_metrics_dir = os.path.join('../..', 'artifacts', 'models_stats', 'dqn')
os.makedirs(model_metrics_dir, exist_ok=True)

train_metrics_name = f'dqn_train_metrics_{environment}_{num_episodes}_{grid_size}.png'
train_metrics_path = os.path.join(model_metrics_dir, train_metrics_name)
agent.save_plots(train_metrics_path)

num_simulations = 100
sim_metrics_name = f'dqn_sim_metrics_{environment}_{num_episodes}_{env.grid_size}_{num_simulations}.jsn'
sim_metrics_path = os.path.join(model_metrics_dir, sim_metrics_name)
compute_metrics(agent, env, sim_metrics_path, num_simulations=num_simulations)

  6%|▌         | 6/100 [00:00<00:03, 24.75it/s]

Snake length: 39, Episode reward: 3329
Snake length: 43, Episode reward: 3621
Snake length: 26, Episode reward: 2150
Snake length: 17, Episode reward: 1328
Snake length: 21, Episode reward: 1707
Snake length: 24, Episode reward: 1991
Snake length: 3, Episode reward: 96


 12%|█▏        | 12/100 [00:00<00:03, 26.73it/s]

Snake length: 40, Episode reward: 3356
Snake length: 33, Episode reward: 2732
Snake length: 27, Episode reward: 2269
Snake length: 17, Episode reward: 1330
Snake length: 22, Episode reward: 1773
Snake length: 2, Episode reward: 16
Snake length: 14, Episode reward: 1098


 19%|█▉        | 19/100 [00:00<00:02, 28.30it/s]

...

 96%|█████████▌| 96/100 [00:03<00:00, 27.89it/s]

Snake length: 23, Episode reward: 1903
Snake length: 3, Episode reward: 111
Snake length: 28, Episode reward: 2320
Snake length: 32, Episode reward: 2661
Snake length: 28, Episode reward: 2312
Snake length: 20, Episode reward: 1576


100%|██████████| 100/100 [00:04<00:00, 24.82it/s]

Snake length: 39, Episode reward: 3319
Snake length: 63, Episode reward: 5462
Snake length: 34, Episode reward: 2853


{'snake_lengths': [39,
  43,
  26,
  17,
  21,
  24,
  3,
  40,
  33,
  27,
  17,
  22,
  2,
  14,
  25,
  42,
  21,
  41,
  21,
  25,
  32,
  34,
  19,
  16,
  23,
  17,
  14,
  32,
  10,
  46,
  34,
  36,
  35,
  23,
  26,
  41,
  11,
  37,
  27,
  39,
  15,
  49,
  25,
  16,
  29,
  12,
  15,
  11,
  30,
  22,
  39,
  7,
  28,
  25,
  21,
  35,
  20,
  25,
  30,
  15,
  23,
  43,
  36,
  22,
  32,
  37,
  42,
  32,
  8,
  36,
  42,
  24,
  30,
  16,
  20,
  16,
  30,
  22,
  35,
  52,
  2,
  33,
  13,
  28,
  21,
  20,
  24,
  40,
  17,
  20,
  23,
  23,
  3,
  28,
  32,
  28,
  20,
  39,
  63,
  34],
 'episode_rewards': [3329,
  3621,
  2150,
  1328,
  1707,
  1991,
  96,
  3356,
  2732,
  2269,
  1330,
  1773,
  16,
  1098,
  2082,
  3597,
  1709,
  3402,
  1677,
  2041,
  2649,
  2877,
  1560,
  1268,
  1886,
  1357,
  1078,
  2642,
  722,
  3969,
  2817,
  3082,
  2872,
  1824,
  2151,
  3531,
  764,
  3153,
  2225,
  3333,
  1133,
  4179,
  2023,
  1284,
  2362,
  922,
  1180,


In [ ]:
if isinstance(env, FullObsSnakeEnv):
    env.interact = True
else:
    env = ParObsSnakeEnv(grid_size=2*grid_size)
    
state = env.reset()
done = False
while not done:
    action, _ = agent.choose_action(state)
    state, reward, done, _ = env.step(action)
    env.render()
    print(f"Reward: {reward}")

Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 76
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 76
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 76
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 76
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 76
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 76
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 76
Reward: 1
Reward: 1
Reward: 1
Rew

: 